# Building the model layers 生成模型层

# What is a neural network 什么是神经网络

神经网络是按层连接的**神经元**的集合。每个神经元都是一个小的计算单元，执行简单的计算来共同解决问题。神经元分为 3 种类型的层：输入层、隐藏层和输出层。隐藏层和输出层包含许多神经元。神经网络模仿人脑处理信息的方式。

# Components of a neural network 神经网络的组成部分

- **activation function** **激活函数** 决定神经元是否应该被激活。神经网络中发生的计算包括应用激活函数。如果神经元激活，则意味着输入很重要。有不同种类的激活函数。选择使用哪个激活函数取决于您想要的输出。激活函数的另一个重要作用是为模型添加非线性。
  - *Binary* 如果函数结果为正，则用于将输出节点设置为 1；如果函数结果为零或负，则将输出节点设置为 0。$f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ 1, & \text{if } x\geq 0\\ \end{cases}}$
  - *Sigmoid* 用于预测输出节点介于 0 和 1 之间的概率。$f(x) = {\large \frac{1}{1+e^{-x}}} $
  - *Tanh* 用于预测输出节点是否在 1 到 -1 之间，用于分类用例。$f(x) = {\large \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}} $
  - *ReLU* (*rectified linear activation function*) 如果函数结果为负，则用于将输出节点设置为 0；如果结果为正，则保持结果值。$f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}$

- **Weights** **权重** 影响我们网络的输出与预期输出值的接近程度。当输入进入神经元时，它会乘以权重值，所得输出要么被观察，要么被传递到神经网络中的下一层。一层中所有神经元的权重被组织成一个张量。

- **Bias** **偏差** 弥补了激活函数的输出与其预期输出之间的差异。低偏差值表明网络对输出形式做出更多假设，而高偏差值对输出形式做出更少假设。

我们可以说，具有weights $W$ 和bias $b$ 的神经网络层的输出 $y$ 的计算为，输入乘以 weights加上bias的总和。 $x = \sum{(weights * inputs) + bias} $，其中 $f(x)$ 是激活函数。

# Build a neural network 构建神经网络

神经网络由对数据执行操作的层/模块组成。[torch.nn](https://pytorch.org/docs/stable/nn.html)命名空间提供了构建您自己的神经网络所需的所有构建块。在PyTorch 中，每个模块都是[nn.Module 的](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)子类。神经网络本身就是一个模块，由其他模块（层）组成。这种嵌套结构允许轻松构建和管理复杂的架构。

在以下部分中，我们将构建一个神经网络，来对 FashionMNIST 数据集中的图像进行分类。

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for training 获取训练设备

我们希望能够在 GPU 等硬件加速器（如果可用）上训练我们的模型。让我们检查一下[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) 或[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html)是否可用，否则我们使用 CPU。

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Define the Class 定义类

我们通过子类化`nn.Module`来定义我们的神经网络。在`__init__`中，初始化神经网络层。每个`nn.Module`子类都在`forward`方法中实现对输入数据的操作。

我们的神经网络由以下部分组成：

- 输入层具有 28x28 或 784 个特征/像素。
- 第一个线性模块采用输入 784 个特征，并将其转换为具有 512 个特征的隐藏层。 
- ReLU 激活函数将应用于转换中。
- 第二个线性模块将第一个隐藏层的 512 个特征作为输入，并将其转换到具有 512 个特征的下一个隐藏层。 
- ReLU 激活函数将应用于转换中。
- 第三个线性模块将 512 个特征作为来自第二个隐藏层的输入，并将这些特征转换到输出层，其中 10 是类的数量。 
- ReLU 激活函数将应用于转换中。

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建`NeuralNetwork` 的一个实例，并将其移动到`device`，并打印其结构。

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


为了使用该model，我们将输入数据传递给它。这将执行model的`forward`以及一些[background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)。不要直接调用`model.forward()`！在输入上调用model会返回一个二维张量，其中 dim=0 对应于每个类的 10 个原始predicted values的每个输出，dim=1 对应于每个输出的各个值。

我们通过，将它传递给`nn.Softmax`模块的实例，来获得prediction probabilities。

In [7]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


#  Weight and Bias 权重和偏差

`nn.Linear` 模块随机初始化每层的权重和偏差，并在内部将值存储在张量中。

In [7]:
print(f"First Linear Weights: {model.linear_relu_stack[0].weight} \n")
print(f"First Linear biases: {model.linear_relu_stack[0].bias} \n")

First Linear Weights: Parameter containing:
tensor([[ 1.8239e-02,  3.2942e-02, -3.2946e-02,  ...,  1.0153e-02,
         -8.4223e-03,  4.5333e-05],
        [ 2.9481e-02,  2.3620e-02,  1.5626e-02,  ...,  2.4684e-02,
         -1.7073e-02,  2.8341e-02],
        [ 3.4109e-02,  2.7542e-02,  2.9627e-02,  ...,  2.7297e-02,
         -6.7970e-03,  2.1646e-02],
        ...,
        [ 5.7005e-03, -9.1041e-03, -3.0561e-02,  ..., -2.5262e-02,
         -3.0317e-02,  3.4043e-02],
        [-3.3178e-02, -8.2961e-03, -3.1984e-02,  ..., -3.2255e-02,
         -3.2282e-02, -3.1796e-02],
        [ 1.5601e-02,  8.5908e-03,  3.3128e-02,  ..., -1.3665e-02,
         -1.2585e-02,  3.0685e-02]], device='cuda:0', requires_grad=True) 

First Linear biases: Parameter containing:
tensor([ 0.0050,  0.0261, -0.0138, -0.0268,  0.0242,  0.0189,  0.0289, -0.0070,
        -0.0327,  0.0235, -0.0189, -0.0075, -0.0213,  0.0308,  0.0036,  0.0250,
        -0.0285, -0.0124, -0.0291,  0.0307,  0.0248,  0.0338, -0.0125,  0.0318,
  

# Model Layers 模型层

让我们分解 FashionMNIST model中的layers。为了说明这一点，我们将采用 3 张大小为 28x28 的图像的小批量样本，看看当我们将其传递到网络时会发生什么。

In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

我们初始化[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)  layer，将每个 2D 28x28 图像转换为 784 个像素值的连续数组，维持小批量维度（在 dim=0 时）。

In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

 [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)是一个使用其存储的权重和偏差对输入应用线性变换的模块。输入层中每个像素的灰度值将连接到隐藏层中的神经元进行计算。用于转换的计算是 ${{weight * input + bias}} $。

In [11]:
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

非线性激活是在模型的输入和输出之间创建复杂映射的原因。它们在线性变换后应用以引入*非线性*，帮助神经网络学习各种现象。

在此模型中，我们在线性层之间使用[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)，但还有其他激活可以在模型中引入非线

ReLU 激活函数获取线性层计算的输出，并将负值替换为零。

Linear output: ${ x = {weight * input + bias}} $。

ReLU: 
$
f(x)= 
\begin{cases}
  0, & \text{if } x < 0\\
  x, & \text{if } x\geq 0\\
\end{cases}
$性。

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0766, -0.0625, -0.2205, -0.0216, -0.2053, -0.2668, -0.0165, -0.2928,
          0.1630,  0.3305,  0.0926,  0.1549, -0.6023,  0.5706,  0.1748,  0.5451,
          0.3832, -0.2599,  0.4527,  0.3424],
        [ 0.4078, -0.3803,  0.1257, -0.4406, -0.2253, -0.1402, -0.2847, -0.3468,
         -0.2220,  0.5142,  0.1939, -0.0116, -0.5433,  0.7497,  0.0924, -0.0852,
          0.2829, -0.4219,  0.0996, -0.0094],
        [ 0.4012, -0.2364, -0.2639, -0.4588, -0.0886, -0.6564, -0.3471, -0.4169,
          0.1507,  0.6246,  0.3622, -0.1566, -0.3550,  0.4770, -0.0493, -0.1288,
          0.3137,  0.1556,  0.0782, -0.1290]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0766, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1630,
         0.3305, 0.0926, 0.1549, 0.0000, 0.5706, 0.1748, 0.5451, 0.3832, 0.0000,
         0.4527, 0.3424],
        [0.4078, 0.0000, 0.1257, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.5142, 0.1939, 0.0000, 0.0000, 0.7497, 0.09

## nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)是模块的有序容器。数据按照定义的相同顺序传递通过所有模块。您可以使用顺序容器来组合一个快速网络，例如`seq_modules`.

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

## nn.Softmax

神经网络的最后一个线性层返回logits （ [-infty, infty] 中的原始值）被传递到 [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)模块。Softmax激活函数用于计算神经网络输出的概率。它仅用于神经网络的输出层。Logits 缩放为值 [0, 1]，表示模型对每个类别的预测概率。`dim`参数指示维度，沿该维度值的总和必须为 1。具有最高概率的节点预测所需的输出。

In [14]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

# Model Parameters 模型参数

神经网络内的许多层都是*参数化的*。在训练期间，优化的相关权重和偏差。子类化`nn.Module`会自动跟踪模型对象中定义的所有字段，并使所有参数都可以使用模型`parameters()`或`named_parameters()`方法进行访问。

在此示例中，我们迭代每个参数，并打印其大小及其值的预览。

In [18]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0337,  0.0320,  0.0072,  ..., -0.0061, -0.0181,  0.0175],
        [ 0.0285,  0.0009,  0.0083,  ...,  0.0137, -0.0123,  0.0303]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0018, -0.0297], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0172,  0.0179,  0.0206,  ..., -0.0175, -0.0417,  0.0410],
        [ 0.0051, -0.0359,  0.0321,  ...,  0.0005,  0.0167,  0.0055]],
       device='cuda:0', grad_fn=<Slice

# 知识检查

PyTorch 中所有神经网络模块的基类为 `torch.nn.Module`

# Further Reading 进一步阅读

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

# References 参考资料

使用 PyTorch 进行机器学习的简介 - Training | Microsoft Learn

[使用 PyTorch 进行机器学习的简介 - Training | Microsoft Learn](https://learn.microsoft.com/zh-cn/training/modules/intro-machine-learning-pytorch/)

# Github

storm-ice/PyTorch_Fundamentals

[storm-ice/PyTorch_Fundamentals](https://github.com/storm-ice/PyTorch_Fundamentals/tree/main)